In [ ]:
import pandas as pd
df = pd.read_csv('data/HAM/HAM10000_metadata')
df.head()

In [34]:
df = df[['image_id', 'dx']]

In [ ]:
df['image_id'] = df['image_id'] + '.jpg'
df.head()

In [36]:
import os
import shutil
folder_obrazow = 'data/HAM/HAM10000'

for index, row in df.iterrows():
    nazwa_pliku = row['image_id']
    klasa = row['dx']
    sciezka_klasy = os.path.join(folder_obrazow, klasa)
    if not os.path.exists(sciezka_klasy):
        os.makedirs(sciezka_klasy)
    zrodlo = os.path.join(folder_obrazow, nazwa_pliku)
    cel = os.path.join(sciezka_klasy, nazwa_pliku)
    shutil.move(zrodlo, cel)

In [37]:
from sklearn.model_selection import train_test_split

def train_val_split(train_dir: str, val_dir: str) -> None:
    os.makedirs(val_dir, exist_ok=True)

    for folder_name in os.listdir(train_dir):
        folder_path = os.path.join(train_dir, folder_name)
        if os.path.isdir(folder_path):
            val_subfolder = os.path.join(val_dir, folder_name)
            os.makedirs(val_subfolder, exist_ok=True)
            files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if
                     os.path.isfile(os.path.join(folder_path, f))]
            train_files, validation_files = train_test_split(files, test_size=0.1, random_state=42)

            for file in validation_files:
                shutil.move(file, os.path.join(val_subfolder, os.path.basename(file)))

In [38]:
train_dir = 'data/HAM/HAM10000'
test_dir = 'data/HAM/HAM10000_test'
train_val_split(train_dir, test_dir)
val_dir = 'data/HAM/HAM10000_val'
train_val_split(train_dir, val_dir)

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
BATCH_SIZE = 64

train_data = image_dataset_from_directory(
        train_dir,
        color_mode='rgb',
        image_size=(100, 75),
        batch_size=BATCH_SIZE,
        label_mode='categorical',
        )

val_data = image_dataset_from_directory(
    val_dir,
    color_mode='rgb',
    image_size=(100, 75),
    batch_size=BATCH_SIZE,
    label_mode='categorical',
)

test_data = image_dataset_from_directory(
    test_dir,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(100, 75),
    label_mode='categorical',
    shuffle=False
)

In [77]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
import tensorflow as tf

input_shape = (100, 75, 3)
num_classes = 7

model = tf.keras.Sequential([
  #  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
 #   tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
   # tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical")
])

model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=input_shape))
model.add(Conv2D(32,kernel_size=(3, 3), activation='relu',padding = 'Same',))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.40))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [78]:
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
earlystop = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

In [79]:
history = model.fit(train_data, epochs=50, validation_data=val_data, callbacks=[earlystop, learning_rate_reduction])

Epoch 1/50


2024-04-22 22:52:44.198303: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_7/dropout_18/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


127/127 [==============================] - 8s 56ms/step - loss: 8.7164 - accuracy: 0.6439 - val_loss: 1.0074 - val_accuracy: 0.6674 - lr: 0.0010
Epoch 2/50
127/127 [==============================] - 7s 55ms/step - loss: 0.9813 - accuracy: 0.6708 - val_loss: 0.9585 - val_accuracy: 0.6707 - lr: 0.0010
Epoch 3/50
127/127 [==============================] - 8s 59ms/step - loss: 0.9402 - accuracy: 0.6684 - val_loss: 0.9144 - val_accuracy: 0.6762 - lr: 0.0010
Epoch 4/50
127/127 [==============================] - 8s 57ms/step - loss: 0.9233 - accuracy: 0.6696 - val_loss: 0.9204 - val_accuracy: 0.6829 - lr: 0.0010
Epoch 5/50
127/127 [==============================] - 8s 56ms/step - loss: 0.9126 - accuracy: 0.6728 - val_loss: 0.8565 - val_accuracy: 0.6939 - lr: 0.0010
Epoch 6/50
127/127 [==============================] - 7s 55ms/step - loss: 0.9017 - accuracy: 0.6730 - val_loss: 0.8508 - val_accuracy: 0.6928 - lr: 0.0010
Epoch 7/50
127/127 [==============================] - 8s 59ms/step - loss: 

In [80]:
model.evaluate(test_data)

16/16 [==============================] - 1s 45ms/step - loss: 0.7569 - accuracy: 0.7184


[0.7569350600242615, 0.7184079885482788]